Обученные модели часто могут ошибаться при анализе данных и, например, давать ложную классификацию. С помощью отображения выходов слоев в векторном пространстве можно выявить степень смешивания разных классов между собой.

Покажем на примере решения задачи классификации ответов пользователей на звонок службы поддержки (напр., "уточняющий вопрос", "хамство" и т.д.)

In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as T
from torch.optim import Adam
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
!pip install git+https://github.com/aimclub/eXplain-NNs &> /dev/null

# !wget https://raw.githubusercontent.com/aimclub/eXplain-NNs/main/requirements.txt
# !pip install -r requirements.txt
! pip install torchmetrics &> /dev/null
! pip install pip install giotto-ph==0.2.2 &> /dev/null
! pip install pip install giotto-tda==0.6.0 &> /dev/null
! pip install umap-learn==0.5.3 &> /dev/null

In [ ]:
from eXNN.visualization import get_random_input, reduce_dim, visualize_layer_manifolds, visualize_recurrent_layer_manifolds

/usr/local/lib/python3.10/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.10/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.10/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Данные представляют собой малую выборку текста, состоящего из ответов живых людей, обзваниваемых оператором. Некоторые экземпляры подобны экземплярам других классов, из-за чего предположительно в признаковом пространстве должно возникнуть сильное смешение.

In [ ]:
!wget -cq https://raw.githubusercontent.com/TheRealGremlin/datasets/main/NLU_data_2.txt

In [ ]:
df = pd.read_csv("NLU_data_2.txt", sep="\t", header=None)

Предобработка текста.

In [ ]:
from keras.preprocessing import   sequence
from keras.preprocessing import   text
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
X_train = df[1]
y_train = df[0]

y_train1 = np.zeros((len(y_train),3))

for i in range(len(y_train)):
  j = y_train[i]-1
  y_train1[i][int(j)] = 1

y_train = np.asarray(y_train).astype('float32')
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)

padded_sequences = pad_sequences(sequences, maxlen=77)

X_train = np.array(padded_sequences)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
#X_train = torch.from_numpy(X_train).type(torch.LongTensor)
print(X_train.shape, y_train.shape, y_train1.shape)

(1522, 1, 77) torch.Size([1522]) (1522, 3)


In [ ]:
inputs = torch.from_numpy(X_train).float()
labels = torch.tensor(y_train1).long()
labels = labels.float()

Простая модель, включающая LSTM и полносвязный слой.

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out


learning_rate = 0.03

In [ ]:
import torch.optim as optim

model = LSTMClassifier(110, 128, 3)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    _, labels1 = torch.max(labels, dim=1)
    return torch.tensor(torch.sum(preds == labels1).item() / len(preds))


In [ ]:
print(inputs[409])

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,  55.,  19.,   9.,   5.,   3.,  34., 732.,  32.,  83., 319.,
         177.,  86.,  32., 154.,   2., 536., 319., 177.,  27.,   4., 224., 123.,
         320.,  61., 221.,   4., 114.]])


Обучаем.

In [ ]:
train_loss = []
train_acc = []

num_epochs=2001
layers = ['lstm']

for epoch in range(num_epochs):
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = loss_function(outputs, labels)

    acc = accuracy(outputs, labels)

    loss.backward()
    optimizer.step()

    if epoch%100==0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}), Accuracy: {acc.item()}')
    if epoch in [0, 2000]:
      visualizations = visualize_recurrent_layer_manifolds(model, mode='umap', data=inputs, layers=layers, labels=labels,stride_mode=1, out_dim=3)
      visualizations[layers[0]].show()
    train_loss.append(loss.item())

Epoch 1/2001, Loss: 1.1295316219329834), Accuracy: 0.29500657320022583


Epoch 101/2001, Loss: 0.8771263360977173), Accuracy: 0.6254927515983582
Epoch 201/2001, Loss: 0.8177137970924377), Accuracy: 0.663600504398346
Epoch 301/2001, Loss: 0.7646445035934448), Accuracy: 0.6931669116020203
Epoch 401/2001, Loss: 0.7248140573501587), Accuracy: 0.716162919998169
Epoch 501/2001, Loss: 0.6842389702796936), Accuracy: 0.7339027523994446
Epoch 601/2001, Loss: 0.6524227261543274), Accuracy: 0.750328540802002
Epoch 701/2001, Loss: 0.626550018787384), Accuracy: 0.762812077999115
Epoch 801/2001, Loss: 0.5986287593841553), Accuracy: 0.7871221899986267
Epoch 901/2001, Loss: 0.5795677304267883), Accuracy: 0.7996057868003845
Epoch 1001/2001, Loss: 0.5649985671043396), Accuracy: 0.8081471920013428
Epoch 1101/2001, Loss: 0.5410983562469482), Accuracy: 0.8232588768005371
Epoch 1201/2001, Loss: 0.5235459208488464), Accuracy: 0.8278580904006958
Epoch 1301/2001, Loss: 0.5072832107543945), Accuracy: 0.8285151124000549
Epoch 1401/2001, Loss: 0.5009310841560364), Accuracy: 0.838370561

На полученных интеративных графиках можно видеть результат визуализации векторного пространства пониженной размерности, где данные в высокой степени наложены из-за наличия общих признаков между отдельными экземплярами (в исходном признаковом пространстве локальная близость двух точек свидетельствует о близком значении некоторых координат, т.е. признаков. Близость в пространстве пониженной размерности также свидетельствует о высокой степени совпадения данных с точки зрения нейронной сети в текущем состоянии). Ввиду того, что в данных много подобных элементов (например, фразы, включающие в себя фразы из других классов), а также большой разницы в длинне векторов, можно наблюдать сильное смешение - в особенности в начале обучения, когда конфигурация параметров нейронной сети практически случайна. На более поздних итерациях наблюдается лучшая сгруппированность данных.